# Análisis de Segmentación de Clientes con K-Means

**Objetivo:** Identificar clusters naturales en la base de clientes mediante técnicas de machine learning no supervisado, optimizando la estrategia de segmentación para acciones comerciales diferenciadas.

**Metodología:** K-Means Clustering con evaluación de k óptimo mediante Elbow Method y Silhouette Score.

---

## 1. Importación de Librerías y Configuración

In [ ]:
# Librerías de manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.decomposition import PCA

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

print("✓ Librerías importadas exitosamente")

## 2. Carga y Exploración Inicial de Datos

In [ ]:
# Cargar datos
df = pd.read_csv('segmentation_data.csv')

print(f"Dimensiones del dataset: {df.shape}")
print(f"\nPrimeras observaciones:")
df.head(10)

In [ ]:
# Información del dataset
print("Información del Dataset:")
print("="*60)
df.info()

print("\n" + "="*60)
print("Valores nulos por columna:")
print(df.isnull().sum())

## 3. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Estadísticas descriptivas
print("Estadísticas Descriptivas - Variables Numéricas")
print("="*80)
df.describe().round(2)

In [ ]:
# Análisis de la segmentación original
print("Distribución de Segmentación Original:")
print("="*60)
segmento_dist = df['Segmento'].value_counts()
print(segmento_dist)
print(f"\nPorcentaje:")
print((segmento_dist / len(df) * 100).round(2))

In [ ]:
# Visualización de distribuciones
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Distribución de Variables Clave', fontsize=16, fontweight='bold')

variables = ['Volumen_Compra', 'Frecuencia_Compra', 'Antigüedad_Meses', 'Ticket_Promedio']
colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']

for idx, (var, color) in enumerate(zip(variables, colors)):
    ax = axes[idx // 2, idx % 2]
    ax.hist(df[var], bins=30, color=color, alpha=0.7, edgecolor='black')
    ax.set_xlabel(var, fontsize=12)
    ax.set_ylabel('Frecuencia', fontsize=12)
    ax.set_title(f'Distribución de {var}', fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Matriz de correlación
correlation_vars = ['Volumen_Compra', 'Frecuencia_Compra', 'Antigüedad_Meses', 'Ticket_Promedio']
correlation_matrix = df[correlation_vars].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación - Variables de Segmentación', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\nAnálisis de Correlaciones:")
print("="*60)
print(correlation_matrix)

In [ ]:
# Boxplots por segmento original
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Distribución de Variables por Segmento Original', fontsize=16, fontweight='bold')

for idx, var in enumerate(variables):
    ax = axes[idx // 2, idx % 2]
    df.boxplot(column=var, by='Segmento', ax=ax, patch_artist=True)
    ax.set_xlabel('Segmento', fontsize=12)
    ax.set_ylabel(var, fontsize=12)
    ax.set_title(f'{var} por Segmento', fontsize=13)
    ax.get_figure().suptitle('')

plt.tight_layout()
plt.show()

## 4. Preparación de Datos para Clustering

In [ ]:
# Selección de features para clustering
features = ['Volumen_Compra', 'Frecuencia_Compra', 'Antigüedad_Meses', 'Ticket_Promedio']
X = df[features].copy()

print(f"Variables seleccionadas para clustering: {features}")
print(f"Shape del dataset: {X.shape}")
print(f"\nPrimeras observaciones:")
X.head()

In [ ]:
# Normalización de datos (crítico para k-means)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=features)

print("Estadísticas después de normalización (media=0, std=1):")
print("="*60)
X_scaled_df.describe().round(3)

## 5. Determinación del Número Óptimo de Clusters

### 5.1 Método del Codo (Elbow Method)

In [ ]:
# Calcular inercia para diferentes valores de k
inertias = []
K_range = range(2, 11)

print("Calculando inercia para k = 2 a 10...")
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    print(f"k={k}: Inercia = {kmeans.inertia_:.2f}")

# Visualización del método del codo
plt.figure(figsize=(12, 6))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=10)
plt.xlabel('Número de Clusters (k)', fontsize=13)
plt.ylabel('Inercia (Within-Cluster Sum of Squares)', fontsize=13)
plt.title('Método del Codo - Determinación de k Óptimo', fontsize=15, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(K_range)

# Añadir anotaciones
for i, (k, inertia) in enumerate(zip(K_range, inertias)):
    plt.annotate(f'{inertia:.0f}', (k, inertia), textcoords="offset points", 
                xytext=(0,10), ha='center', fontsize=9)

plt.tight_layout()
plt.show()

### 5.2 Silhouette Score

In [ ]:
# Calcular Silhouette Score para diferentes valores de k
silhouette_scores = []

print("Calculando Silhouette Score para k = 2 a 10...")
print("="*60)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=300)
    cluster_labels = kmeans.fit_predict(X_scaled)
    silhouette_avg = silhouette_score(X_scaled, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"k={k}: Silhouette Score = {silhouette_avg:.4f}")

# Identificar el k óptimo
optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"\n{'='*60}")
print(f"K ÓPTIMO según Silhouette Score: {optimal_k}")
print(f"Mejor Silhouette Score: {max(silhouette_scores):.4f}")
print(f"{'='*60}")

In [ ]:
# Visualización del Silhouette Score
plt.figure(figsize=(12, 6))
plt.plot(K_range, silhouette_scores, 'ro-', linewidth=2, markersize=10)
plt.xlabel('Número de Clusters (k)', fontsize=13)
plt.ylabel('Silhouette Score', fontsize=13)
plt.title('Silhouette Score por Número de Clusters', fontsize=15, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(K_range)

# Marcar el k óptimo
plt.axvline(x=optimal_k, color='green', linestyle='--', linewidth=2, 
            label=f'K óptimo = {optimal_k}')
plt.legend(fontsize=12)

# Añadir anotaciones
for i, (k, score) in enumerate(zip(K_range, silhouette_scores)):
    plt.annotate(f'{score:.3f}', (k, score), textcoords="offset points", 
                xytext=(0,10), ha='center', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Gráfico comparativo: Inercia vs Silhouette Score
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Inercia
ax1.plot(K_range, inertias, 'bo-', linewidth=2, markersize=10)
ax1.set_xlabel('Número de Clusters (k)', fontsize=12)
ax1.set_ylabel('Inercia', fontsize=12)
ax1.set_title('Método del Codo', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_xticks(K_range)

# Silhouette
ax2.plot(K_range, silhouette_scores, 'ro-', linewidth=2, markersize=10)
ax2.set_xlabel('Número de Clusters (k)', fontsize=12)
ax2.set_ylabel('Silhouette Score', fontsize=12)
ax2.set_title('Silhouette Score', fontsize=14, fontweight='bold')
ax2.axvline(x=optimal_k, color='green', linestyle='--', linewidth=2)
ax2.grid(True, alpha=0.3)
ax2.set_xticks(K_range)

plt.tight_layout()
plt.show()

## 6. Aplicación de K-Means con K Óptimo

In [ ]:
# Entrenar modelo con k óptimo
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=20, max_iter=500)
cluster_labels = kmeans_final.fit_predict(X_scaled)

# Agregar clusters al dataframe original
df['Cluster_KMeans'] = cluster_labels

print(f"Modelo K-Means entrenado con k={optimal_k}")
print(f"Inercia final: {kmeans_final.inertia_:.2f}")
print(f"Silhouette Score final: {silhouette_score(X_scaled, cluster_labels):.4f}")
print(f"\nIteraciones hasta convergencia: {kmeans_final.n_iter_}")

In [ ]:
# Distribución de clusters
cluster_dist = df['Cluster_KMeans'].value_counts().sort_index()

print("\nDistribución de Clientes por Cluster:")
print("="*60)
for cluster_id, count in cluster_dist.items():
    percentage = (count / len(df)) * 100
    print(f"Cluster {cluster_id}: {count} clientes ({percentage:.2f}%)")

# Visualización
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Barras
ax1.bar(cluster_dist.index, cluster_dist.values, color=sns.color_palette("husl", optimal_k), 
        edgecolor='black', alpha=0.7)
ax1.set_xlabel('Cluster', fontsize=12)
ax1.set_ylabel('Número de Clientes', fontsize=12)
ax1.set_title('Distribución de Clientes por Cluster', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Añadir valores en las barras
for i, (cluster_id, count) in enumerate(cluster_dist.items()):
    percentage = (count / len(df)) * 100
    ax1.text(cluster_id, count, f'{count}\n({percentage:.1f}%)', 
            ha='center', va='bottom', fontsize=11, fontweight='bold')

# Pie chart
colors = sns.color_palette("husl", optimal_k)
wedges, texts, autotexts = ax2.pie(cluster_dist.values, labels=[f'Cluster {i}' for i in cluster_dist.index],
                                     autopct='%1.1f%%', startangle=90, colors=colors,
                                     textprops={'fontsize': 11, 'fontweight': 'bold'})
ax2.set_title('Proporción de Clientes por Cluster', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Análisis de Perfiles de Clusters

In [ ]:
# Estadísticas descriptivas por cluster
cluster_profiles = df.groupby('Cluster_KMeans')[features].agg(['mean', 'median', 'std', 'min', 'max'])

print("Perfiles de Clusters - Estadísticas Descriptivas")
print("="*100)
cluster_profiles.round(2)

In [ ]:
# Valores promedio por cluster (tabla simplificada)
cluster_means = df.groupby('Cluster_KMeans')[features].mean().round(2)

print("\nValores Promedio por Cluster:")
print("="*80)
print(cluster_means)

# Crear una tabla visual
fig, ax = plt.subplots(figsize=(12, 6))
ax.axis('tight')
ax.axis('off')

table_data = cluster_means.reset_index().values
col_labels = ['Cluster'] + features

table = ax.table(cellText=table_data, colLabels=col_labels, cellLoc='center', loc='center',
                colColours=['lightgray']*len(col_labels))
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2)

plt.title('Valores Promedio por Cluster - Resumen', fontsize=15, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de radar para perfiles de clusters (normalizado)
from math import pi

# Normalizar los valores entre 0 y 1 para mejor visualización
cluster_means_normalized = cluster_means.copy()
for col in cluster_means.columns:
    min_val = cluster_means[col].min()
    max_val = cluster_means[col].max()
    cluster_means_normalized[col] = (cluster_means[col] - min_val) / (max_val - min_val)

# Configuración del gráfico de radar
categories = features
N = len(categories)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(12, 12), subplot_kw=dict(projection='polar'))
colors = sns.color_palette("husl", optimal_k)

for idx, cluster_id in enumerate(cluster_means_normalized.index):
    values = cluster_means_normalized.loc[cluster_id].values.tolist()
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=f'Cluster {cluster_id}', 
           color=colors[idx])
    ax.fill(angles, values, alpha=0.15, color=colors[idx])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, size=12)
ax.set_ylim(0, 1)
ax.set_yticks([0.25, 0.5, 0.75, 1.0])
ax.set_yticklabels(['0.25', '0.5', '0.75', '1.0'], size=10)
ax.grid(True, linestyle='--', alpha=0.7)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)
ax.set_title('Perfiles de Clusters - Gráfico Radar (Normalizado)', 
            fontsize=15, fontweight='bold', pad=30)

plt.tight_layout()
plt.show()

In [ ]:
# Heatmap de perfiles de clusters
plt.figure(figsize=(12, 6))
sns.heatmap(cluster_means_normalized, annot=cluster_means.values, fmt='.0f', 
            cmap='YlOrRd', cbar_kws={'label': 'Valor Normalizado (0-1)'},
            linewidths=1, linecolor='black')
plt.title('Heatmap de Perfiles de Clusters (Valores Reales Anotados)', 
         fontsize=14, fontweight='bold', pad=15)
plt.xlabel('Variables', fontsize=12)
plt.ylabel('Cluster', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Comparación cluster por cluster
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Distribución de Variables por Cluster K-Means', fontsize=16, fontweight='bold')

for idx, var in enumerate(features):
    ax = axes[idx // 2, idx % 2]
    df.boxplot(column=var, by='Cluster_KMeans', ax=ax, patch_artist=True)
    ax.set_xlabel('Cluster', fontsize=12)
    ax.set_ylabel(var, fontsize=12)
    ax.set_title(f'{var} por Cluster', fontsize=13)
    ax.get_figure().suptitle('')
    
plt.tight_layout()
plt.show()

## 8. Visualización de Clusters en 2D (PCA)

In [ ]:
# Aplicar PCA para reducir a 2 dimensiones
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

# Varianza explicada
explained_variance = pca.explained_variance_ratio_
print(f"Varianza explicada por componente:")
print(f"PC1: {explained_variance[0]:.4f} ({explained_variance[0]*100:.2f}%)")
print(f"PC2: {explained_variance[1]:.4f} ({explained_variance[1]*100:.2f}%)")
print(f"Total varianza explicada: {sum(explained_variance):.4f} ({sum(explained_variance)*100:.2f}%)")

In [ ]:
# Visualización de clusters en espacio PCA
plt.figure(figsize=(14, 10))

# Scatter plot por cluster
colors = sns.color_palette("husl", optimal_k)
for cluster_id in range(optimal_k):
    mask = cluster_labels == cluster_id
    plt.scatter(X_pca[mask, 0], X_pca[mask, 1], 
               c=[colors[cluster_id]], label=f'Cluster {cluster_id}',
               alpha=0.6, s=100, edgecolors='black', linewidth=0.5)

# Centroides en espacio PCA
centroids_pca = pca.transform(kmeans_final.cluster_centers_)
plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], 
           c='red', marker='X', s=500, edgecolors='black', linewidth=2,
           label='Centroides', zorder=10)

plt.xlabel(f'PC1 ({explained_variance[0]*100:.2f}% varianza)', fontsize=13)
plt.ylabel(f'PC2 ({explained_variance[1]*100:.2f}% varianza)', fontsize=13)
plt.title(f'Visualización de Clusters en Espacio PCA (k={optimal_k})', 
         fontsize=15, fontweight='bold')
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Contribution de variables a los componentes principales
components_df = pd.DataFrame(
    pca.components_,
    columns=features,
    index=['PC1', 'PC2']
)

print("Contribución de Variables a Componentes Principales:")
print("="*70)
print(components_df.round(3))

# Visualización de cargas
plt.figure(figsize=(12, 6))
sns.heatmap(components_df, annot=True, cmap='coolwarm', center=0, 
            fmt='.3f', linewidths=1, cbar_kws={'label': 'Carga'})
plt.title('Matriz de Cargas - Componentes Principales', fontsize=14, fontweight='bold')
plt.xlabel('Variables', fontsize=12)
plt.ylabel('Componente', fontsize=12)
plt.tight_layout()
plt.show()

## 9. Análisis de Silhouette Detallado

In [ ]:
# Silhouette plot detallado
silhouette_vals = silhouette_samples(X_scaled, cluster_labels)
silhouette_avg = silhouette_score(X_scaled, cluster_labels)

fig, ax = plt.subplots(figsize=(12, 8))
y_lower = 10
colors = sns.color_palette("husl", optimal_k)

for i in range(optimal_k):
    cluster_silhouette_vals = silhouette_vals[cluster_labels == i]
    cluster_silhouette_vals.sort()
    
    size_cluster_i = cluster_silhouette_vals.shape[0]
    y_upper = y_lower + size_cluster_i
    
    ax.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_silhouette_vals,
                     facecolor=colors[i], edgecolor=colors[i], alpha=0.7)
    
    ax.text(-0.05, y_lower + 0.5 * size_cluster_i, f'C{i}', 
           fontsize=12, fontweight='bold')
    
    y_lower = y_upper + 10

ax.axvline(x=silhouette_avg, color="red", linestyle="--", linewidth=2,
          label=f'Silhouette promedio: {silhouette_avg:.3f}')
ax.set_xlabel('Coeficiente de Silhouette', fontsize=13)
ax.set_ylabel('Cluster', fontsize=13)
ax.set_title(f'Análisis de Silhouette para K-Means (k={optimal_k})', 
            fontsize=15, fontweight='bold')
ax.set_yticks([])
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# Silhouette score por cluster
print("\nSilhouette Score por Cluster:")
print("="*60)
for i in range(optimal_k):
    cluster_silhouette_avg = silhouette_vals[cluster_labels == i].mean()
    print(f"Cluster {i}: {cluster_silhouette_avg:.4f}")

## 10. Comparación con Segmentación Original

In [ ]:
# Tabla de contingencia
contingency_table = pd.crosstab(df['Segmento'], df['Cluster_KMeans'], margins=True)

print("Tabla de Contingencia: Segmentación Original vs K-Means")
print("="*70)
print(contingency_table)

In [ ]:
# Heatmap de comparación
contingency_normalized = pd.crosstab(df['Segmento'], df['Cluster_KMeans'], normalize='index')

plt.figure(figsize=(10, 6))
sns.heatmap(contingency_normalized, annot=True, fmt='.2%', cmap='YlGnBu', 
            linewidths=1, cbar_kws={'label': 'Proporción'})
plt.title('Comparación: Segmentación Original vs Clusters K-Means (% por fila)', 
         fontsize=14, fontweight='bold')
plt.xlabel('Cluster K-Means', fontsize=12)
plt.ylabel('Segmento Original', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
# Visualización lado a lado: Original vs K-Means en PCA
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# Segmentación original
segmentos_unicos = df['Segmento'].unique()
colors_original = sns.color_palette("Set2", len(segmentos_unicos))
for idx, seg in enumerate(segmentos_unicos):
    mask = df['Segmento'] == seg
    ax1.scatter(X_pca[mask, 0], X_pca[mask, 1], 
               c=[colors_original[idx]], label=seg,
               alpha=0.6, s=100, edgecolors='black', linewidth=0.5)
ax1.set_xlabel(f'PC1 ({explained_variance[0]*100:.2f}%)', fontsize=12)
ax1.set_ylabel(f'PC2 ({explained_variance[1]*100:.2f}%)', fontsize=12)
ax1.set_title('Segmentación Original', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Clusters K-Means
colors_kmeans = sns.color_palette("husl", optimal_k)
for cluster_id in range(optimal_k):
    mask = cluster_labels == cluster_id
    ax2.scatter(X_pca[mask, 0], X_pca[mask, 1], 
               c=[colors_kmeans[cluster_id]], label=f'Cluster {cluster_id}',
               alpha=0.6, s=100, edgecolors='black', linewidth=0.5)
ax2.scatter(centroids_pca[:, 0], centroids_pca[:, 1], 
           c='red', marker='X', s=400, edgecolors='black', linewidth=2,
           label='Centroides', zorder=10)
ax2.set_xlabel(f'PC1 ({explained_variance[0]*100:.2f}%)', fontsize=12)
ax2.set_ylabel(f'PC2 ({explained_variance[1]*100:.2f}%)', fontsize=12)
ax2.set_title(f'Clusters K-Means (k={optimal_k})', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Interpretación de Clusters y Recomendaciones

In [ ]:
# Generar descripción automática de clusters
print("PERFILES DE CLUSTERS - INTERPRETACIÓN")
print("="*80)
print()

for cluster_id in range(optimal_k):
    cluster_data = df[df['Cluster_KMeans'] == cluster_id]
    n_clientes = len(cluster_data)
    pct_clientes = (n_clientes / len(df)) * 100
    
    print(f"CLUSTER {cluster_id}:")
    print("-" * 80)
    print(f"Tamaño: {n_clientes} clientes ({pct_clientes:.1f}% del total)")
    print()
    print("Características promedio:")
    for feature in features:
        mean_val = cluster_data[feature].mean()
        overall_mean = df[feature].mean()
        diff_pct = ((mean_val - overall_mean) / overall_mean) * 100
        indicator = "↑" if diff_pct > 10 else "↓" if diff_pct < -10 else "≈"
        print(f"  {indicator} {feature}: {mean_val:.2f} ({diff_pct:+.1f}% vs promedio general)")
    
    # Relación con segmentación original
    print("\nComposición según segmentación original:")
    seg_dist = cluster_data['Segmento'].value_counts()
    for seg, count in seg_dist.items():
        pct = (count / n_clientes) * 100
        print(f"  - {seg}: {count} ({pct:.1f}%)")
    
    print("\n" + "="*80 + "\n")

In [ ]:
# Exportar resultados
df_export = df[['ID_Cliente', 'Nombre', 'Volumen_Compra', 'Frecuencia_Compra', 
                'Antigüedad_Meses', 'Ticket_Promedio', 'Segmento', 'Cluster_KMeans']].copy()

# Añadir características del cluster
df_export['Cluster_Size'] = df_export['Cluster_KMeans'].map(cluster_dist)
df_export['Cluster_Pct'] = (df_export['Cluster_Size'] / len(df) * 100).round(2)

# Añadir componentes PCA
df_export['PCA_1'] = X_pca[:, 0]
df_export['PCA_2'] = X_pca[:, 1]

# Guardar archivo
output_file = 'resultados_clustering_kmeans.csv'
df_export.to_csv(output_file, index=False)
print(f"✓ Resultados exportados a: {output_file}")
print(f"  Registros: {len(df_export)}")
print(f"  Columnas: {len(df_export.columns)}")

## 12. Resumen Ejecutivo

In [ ]:
print("\n" + "="*80)
print("RESUMEN EJECUTIVO - ANÁLISIS DE CLUSTERING K-MEANS")
print("="*80)
print()

print("METODOLOGÍA:")
print("-" * 80)
print(f"• Algoritmo: K-Means Clustering")
print(f"• Variables utilizadas: {', '.join(features)}")
print(f"• Técnica de normalización: StandardScaler")
print(f"• Método de optimización: Elbow Method + Silhouette Score")
print()

print("RESULTADOS PRINCIPALES:")
print("-" * 80)
print(f"• Número óptimo de clusters (k): {optimal_k}")
print(f"• Silhouette Score: {silhouette_score(X_scaled, cluster_labels):.4f}")
print(f"• Inercia (WCSS): {kmeans_final.inertia_:.2f}")
print(f"• Varianza explicada (PCA 2D): {sum(explained_variance)*100:.2f}%")
print()

print("DISTRIBUCIÓN DE CLUSTERS:")
print("-" * 80)
for cluster_id in range(optimal_k):
    count = cluster_dist[cluster_id]
    pct = (count / len(df)) * 100
    print(f"• Cluster {cluster_id}: {count} clientes ({pct:.2f}%)")
print()

print("INSIGHTS CLAVE:")
print("-" * 80)
print("• El algoritmo ha identificado patrones diferenciados en el comportamiento")
print("  de compra de los clientes")
print(f"• La correlación más fuerte se observa entre:")
corr_max = correlation_matrix.abs().unstack().sort_values(ascending=False)
corr_max = corr_max[corr_max < 1.0]
if len(corr_max) > 0:
    top_corr = corr_max.iloc[0]
    top_vars = corr_max.index[0]
    print(f"  {top_vars[0]} y {top_vars[1]} (r={top_corr:.3f})")
print(f"• Los clusters muestran separación clara en el espacio PCA")
print()

print("RECOMENDACIONES:")
print("-" * 80)
print("1. Validar clusters con equipos de negocio y marketing")
print("2. Desarrollar estrategias diferenciadas por cluster")
print("3. Implementar scoring de nuevos clientes usando centroides")
print("4. Monitorear migración de clientes entre clusters (early warning)")
print("5. Considerar variables adicionales para refinamiento (RFM, Canal, Producto)")
print()

print("PRÓXIMOS PASOS:")
print("-" * 80)
print("• Asignar nombres de negocio a cada cluster")
print("• Crear dashboards interactivos para seguimiento")
print("• Implementar modelos predictivos (CLV, Churn) por cluster")
print("• Diseñar campaigns y ofertas personalizadas")
print("• Establecer KPIs y OKRs por segmento")
print()

print("="*80)
print(f"Análisis completado: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## 13. Exploración Adicional (Opcional)

In [ ]:
# Análisis de estabilidad - ejecutar k-means múltiples veces
n_iterations = 10
stability_scores = []

print(f"Evaluando estabilidad del clustering (n={n_iterations} iteraciones)...")
for i in range(n_iterations):
    kmeans_temp = KMeans(n_clusters=optimal_k, random_state=i, n_init=10)
    labels_temp = kmeans_temp.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels_temp)
    stability_scores.append(score)

print(f"\nSilhouette Score - Estadísticas de estabilidad:")
print(f"  Media: {np.mean(stability_scores):.4f}")
print(f"  Std: {np.std(stability_scores):.4f}")
print(f"  Min: {np.min(stability_scores):.4f}")
print(f"  Max: {np.max(stability_scores):.4f}")
print(f"  CV: {(np.std(stability_scores)/np.mean(stability_scores)*100):.2f}%")

# Visualización
plt.figure(figsize=(10, 6))
plt.plot(range(1, n_iterations+1), stability_scores, 'bo-', linewidth=2, markersize=8)
plt.axhline(y=np.mean(stability_scores), color='r', linestyle='--', 
           label=f'Media: {np.mean(stability_scores):.4f}')
plt.fill_between(range(1, n_iterations+1), 
                np.mean(stability_scores) - np.std(stability_scores),
                np.mean(stability_scores) + np.std(stability_scores),
                alpha=0.2, color='red')
plt.xlabel('Iteración', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.title('Análisis de Estabilidad del Clustering', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Matriz de distancias entre centroides
from scipy.spatial.distance import pdist, squareform

centroids = kmeans_final.cluster_centers_
distances = squareform(pdist(centroids, metric='euclidean'))
distance_df = pd.DataFrame(distances, 
                          columns=[f'C{i}' for i in range(optimal_k)],
                          index=[f'C{i}' for i in range(optimal_k)])

print("Matriz de Distancias Euclidianas entre Centroides:")
print("="*60)
print(distance_df.round(3))

plt.figure(figsize=(10, 8))
sns.heatmap(distance_df, annot=True, fmt='.2f', cmap='viridis', 
            square=True, linewidths=1, cbar_kws={'label': 'Distancia Euclidiana'})
plt.title('Distancias entre Centroides de Clusters', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

---

## Conclusiones

Este análisis de clustering con K-Means ha permitido:

1. **Identificar el número óptimo de clusters** mediante técnicas complementarias (Elbow Method + Silhouette Score)
2. **Caracterizar perfiles diferenciados** de clientes basados en comportamiento de compra
3. **Validar la segmentación** mediante múltiples métricas de calidad (inercia, silhouette, PCA)
4. **Comparar** con la segmentación previa para identificar oportunidades de refinamiento
5. **Generar insights accionables** para estrategias comerciales diferenciadas

### Valor de Negocio

- **Personalización**: Desarrollo de ofertas y comunicaciones específicas por cluster
- **Eficiencia**: Optimización de inversión comercial enfocándose en segmentos de alto valor
- **Retención**: Identificación temprana de patrones de riesgo
- **Crecimiento**: Estrategias para migrar clientes a segmentos superiores

### Limitaciones y Consideraciones

- K-Means asume clusters esféricos y de tamaño similar
- Sensible a outliers y escala de variables (mitigado con normalización)
- Requiere especificar k de antemano (resuelto con métodos de optimización)
- Clustering no supervisado - requiere validación de negocio

---

**Autor**: Análisis de Clustering K-Means  
**Fecha**: 2025  
**Herramientas**: Python 3.x, scikit-learn, pandas, matplotlib, seaborn